In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
import random
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from numpy.random import seed
import pandas as pd
import matplotlib as mat
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import cv2
from tensorflow.random import set_seed
import warnings
%matplotlib inline

In [5]:
pd.options.display.max_colwidth = 100
seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed(42)
warnings.filterwarnings('ignore')

In [6]:
IMG_SIZE = 224
BATCH = 32
SEED = 42

In [7]:
main_path = "/content/drive/MyDrive/chest_xray"

train_path = os.path.join(main_path,"train")

test_path=os.path.join(main_path,"test")

train_normal = glob.glob(train_path+"/NORMAL/*.jpeg")

train_pneumonia = glob.glob(train_path+"/PNEUMONIA/*.jpeg")

test_normal = glob.glob(test_path+"/NORMAL/*.jpeg")

test_pneumonia = glob.glob(test_path+"/PNEUMONIA/*.jpeg")

In [8]:
train_list = [x for x in train_normal]
train_list.extend([x for x in train_pneumonia])

df_train = pd.DataFrame(np.concatenate([['Normal']*len(train_normal) , ['Pneumonia']*len(train_pneumonia)]), columns = ['class'])
df_train['image'] = [x for x in train_list]

test_list = [x for x in test_normal]
test_list.extend([x for x in test_pneumonia])

df_test = pd.DataFrame(np.concatenate([['Normal']*len(test_normal) , ['Pneumonia']*len(test_pneumonia)]), columns = ['class'])
df_test['image'] = [x for x in test_list]

In [11]:
train_df, val_df = train_test_split(df_train, test_size = 0.20, random_state = SEED, stratify = df_train['class'])

In [12]:
train_df

,class,image
3566,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-4799529-0001.jpeg
2866,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-9530903-0004.jpeg
2681,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-864208-0002.jpeg
1199,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-2386123-0001.jpeg
4619,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-1797366-0001.jpeg
...,...,...
3476,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-4932029-0003.jpeg
678,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-7824011-0001.jpeg
1560,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/VIRUS-52163-0003.jpeg
2769,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-9424776-0001.jpeg


In [13]:
val_df

,class,image
2945,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-9835318-0001.jpeg
4878,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-3362849-0001.jpeg
3177,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/VIRUS-2427242-0005.jpeg
972,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-9756687-0001.jpeg
3059,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/VIRUS-1591633-0002.jpeg
...,...,...
253,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-5075834-0001.jpeg
4315,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-7847892-0004.jpeg
687,Normal,/content/drive/MyDrive/chest_xray/train/NORMAL/NORMAL-7953615-0001.jpeg
3417,Pneumonia,/content/drive/MyDrive/chest_xray/train/PNEUMONIA/BACTERIA-4352107-0001.jpeg


In [14]:
train_datagen = ImageDataGenerator(rescale=1/255.,
                                  zoom_range = 0.1,

                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1)

val_datagen = ImageDataGenerator(rescale=1/255.)

ds_train = train_datagen.flow_from_dataframe(train_df,

                                             x_col = 'image',
                                             y_col = 'class',
                                             target_size = (IMG_SIZE, IMG_SIZE),
                                             class_mode = 'binary',
                                             batch_size = BATCH,
                                             seed = SEED)

ds_val = val_datagen.flow_from_dataframe(val_df,

                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            class_mode = 'binary',
                                            batch_size = BATCH,
                                            seed = SEED)

ds_test = val_datagen.flow_from_dataframe(df_test,

                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            class_mode = 'binary',
                                            batch_size = 1,
                                            shuffle = False)

Found 4185 validated image filenames belonging to 2 classes.
Found 1047 validated image filenames belonging to 2 classes.
Found 661 validated image filenames belonging to 2 classes.


In [15]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    min_delta=1e-7,
    restore_best_weights=True,
)

plateau = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor = 0.2,
    patience = 2,
    min_delt = 1e-7,
    cooldown = 0,
    verbose = 1
)

In [16]:
def get_model():

    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))


    x = layers.Conv2D(filters=16, kernel_size=3, padding='valid')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Dropout(0.2)(x)


    x = layers.Conv2D(filters=32, kernel_size=3, padding='valid')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Dropout(0.2)(x)


    x = layers.Conv2D(filters=64, kernel_size=3, padding='valid')(x)
    x = layers.Conv2D(filters=64, kernel_size=3, padding='valid')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Dropout(0.4)(x)


    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    output = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=[inputs], outputs=output)

    return model

In [17]:
keras.backend.clear_session()

model = get_model()
model.compile(loss='binary_crossentropy'
              , optimizer = keras.optimizers.Adam(learning_rate=3e-5), metrics='binary_accuracy')

In [18]:
history = model.fit(ds_train,
          batch_size = BATCH, epochs = 5,
          validation_data=ds_val,
          callbacks=[early_stopping, plateau],
          steps_per_epoch=(len(train_df)/BATCH),
          validation_steps=(len(val_df)/BATCH));

Epoch 1/5
130/130 [==============================] - 1024s 8s/step - loss: 0.4755 - binary_accuracy: 0.7990 - val_loss: 1.1304 - val_binary_accuracy: 0.7421 - lr: 3.0000e-05
Epoch 2/5
130/130 [==============================] - 124s 952ms/step - loss: 0.3130 - binary_accuracy: 0.8636 - val_loss: 1.8536 - val_binary_accuracy: 0.7421 - lr: 3.0000e-05
Epoch 3/5
131/130 [==============================] - ETA: 0s - loss: 0.2562 - binary_accuracy: 0.8894
Epoch 3: ReduceLROnPlateau reducing learning rate to 5.9999998484272515e-06.
130/130 [==============================] - 119s 913ms/step - loss: 0.2562 - binary_accuracy: 0.8894 - val_loss: 1.5512 - val_binary_accuracy: 0.7421 - lr: 3.0000e-05
Epoch 4/5
130/130 [==============================] - 125s 959ms/step - loss: 0.2252 - binary_accuracy: 0.9027 - val_loss: 1.1258 - val_binary_accuracy: 0.7421 - lr: 6.0000e-06
Epoch 5/5
130/130 [==============================] - 115s 878ms/step - loss: 0.2241 - binary_accuracy: 0.9097 - val_loss: 0.2358 

In [21]:
score = model.evaluate(ds_val, steps = len(val_df)/BATCH, verbose = 0)

In [22]:
print('Training Loss for Custom CNN: ',score[0])
print('Training Accuracy for Custom CNN: ',score[1]*100, '%')

Training Loss for Custom CNN:  0.2358359694480896
Training Accuracy for Custom CNN:  88.53868246078491 %


In [23]:
score = model.evaluate(ds_test, steps = len(df_test), verbose = 0)

In [24]:
print('Test Loss for Custom CNN: ',score[0])
print('Test Accuracy for Custom CNN: ',score[1]*100, '%')

Test Loss for Custom CNN:  0.6424005031585693
Test Accuracy for Custom CNN:  71.86081409454346 %
